### Seamless Bay Area Tweet Analysis: Part 1, Data Wrangling


The goal of this project is to analyze the twitter account of the nonprofit group Seamless Bay Area and determine what makes up the most high-impact tweet as measured by engagements.

In part one we wrangle/munge the data to get it into a useable form for EDA.

In [1]:
# Load necessary libraries
import pandas as pd
import numpy as np
import statistics as stat
import re
from sklearn import linear_model
from sklearn.linear_model import LinearRegression
import os

I manually downloaded the data from twitter, in the form of 48 seperate CSV's (one for each month covered) and put them in a single folder. First, let's get a list of all the files.

In [36]:
labels = os.listdir('/Users/grahamsmith/Documents/SpringboardWork/Seamless_Twitter_Analysis/tweets')

After removing hidden files, we loop through all the filenames and append them into a single dataframe. There are only ~2200 tweets, so a more memory efficient data structure isn't necessary.

In [49]:
#remove hidden files
temp = []
for x in labels:
    if x != '.DS_Store':
        temp.append(x)
labels = temp

#each month of tweets is in a seperate .csv, so we combine them together
csvs = []
for x in labels:
   csvs.append(pd.read_csv('/Users/grahamsmith/Documents/SpringboardWork/Seamless_Twitter_Analysis/tweets/' + x ))
df = pd.concat(csvs)

Let's take a look at the data to see what we've got.

In [50]:
df.head()

,Tweet id,Tweet permalink,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,...,promoted hashtag clicks,promoted detail expands,promoted permalink clicks,promoted app opens,promoted app installs,promoted follows,promoted email tweet,promoted dial phone,promoted media views,promoted media engagements
0,1056244576426516480,https://twitter.com/seamlessbayarea/status/105...,@MTCBATA is looking for a new Executive Direct...,2018-10-27 18:01 +0000,124.0,5.0,0.040323,0.0,0.0,0.0,...,-,-,-,-,-,-,-,-,-,-
1,1055827476406927360,https://twitter.com/seamlessbayarea/status/105...,Ultimate seamlessness. https://t.co/CdCLrg2o6a,2018-10-26 14:24 +0000,345.0,10.0,0.028986,0.0,0.0,1.0,...,-,-,-,-,-,-,-,-,-,-
2,1055647302742814720,https://twitter.com/seamlessbayarea/status/105...,Help Nix Prop 6! Save funding for more seamles...,2018-10-26 02:28 +0000,994.0,19.0,0.019115,4.0,0.0,5.0,...,-,-,-,-,-,-,-,-,-,-
3,1054877583471390720,https://twitter.com/seamlessbayarea/status/105...,It doesn't have to be this way! Let's get to f...,2018-10-23 23:29 +0000,792.0,7.0,0.008838,2.0,0.0,4.0,...,-,-,-,-,-,-,-,-,-,-
4,1054872404407726080,https://twitter.com/seamlessbayarea/status/105...,And then come say hi at next month’s @SPUR_Urb...,2018-10-23 23:09 +0000,532.0,3.0,0.005639,0.0,0.0,1.0,...,-,-,-,-,-,-,-,-,-,-


Looks like we expected it to, but some of those columns look like they might be empty. Let's check them.

In [51]:
df.columns

Index(['Tweet id', 'Tweet permalink', 'Tweet text', 'time', 'impressions',
       'engagements', 'engagement rate', 'retweets', 'replies', 'likes',
       'user profile clicks', 'url clicks', 'hashtag clicks', 'detail expands',
       'permalink clicks', 'app opens', 'app installs', 'follows',
       'email tweet', 'dial phone', 'media views', 'media engagements',
       'promoted impressions', 'promoted engagements',
       'promoted engagement rate', 'promoted retweets', 'promoted replies',
       'promoted likes', 'promoted user profile clicks', 'promoted url clicks',
       'promoted hashtag clicks', 'promoted detail expands',
       'promoted permalink clicks', 'promoted app opens',
       'promoted app installs', 'promoted follows', 'promoted email tweet',
       'promoted dial phone', 'promoted media views',
       'promoted media engagements'],
      dtype='object')

We loop through every column and check that each value is equal to the string "-".

In [52]:
# Every column from 23 to the end is a "promoted" column, which we suspect has no data
promoted_cols = df.iloc[:,22:]

# Loop through the columns, comparing the sum of all comparisons to the columns length
temp1 = []
for y in range(len(promoted_cols.columns)):
    temp = []
    for x in range(len(promoted_cols)):
        temp.append(promoted_cols.iloc[x,y] == '-')
    temp1.append(sum(temp) == len(promoted_cols))
sum(temp1) == len(promoted_cols.columns)

True

Perfect, it looks like all those columns are empty and we can safetly drop them. We're also going to drop the first two columns because Tweet ID and Permalink are just identifiers we won't be using.

In [53]:
#remove columns with no/irrelevant data
df = df.iloc[:,2:12]

df.head()

,Tweet text,time,impressions,engagements,engagement rate,retweets,replies,likes,user profile clicks,url clicks
0,@MTCBATA is looking for a new Executive Direct...,2018-10-27 18:01 +0000,124.0,5.0,0.040323,0.0,0.0,0.0,1.0,4.0
1,Ultimate seamlessness. https://t.co/CdCLrg2o6a,2018-10-26 14:24 +0000,345.0,10.0,0.028986,0.0,0.0,1.0,0.0,9.0
2,Help Nix Prop 6! Save funding for more seamles...,2018-10-26 02:28 +0000,994.0,19.0,0.019115,4.0,0.0,5.0,3.0,5.0
3,It doesn't have to be this way! Let's get to f...,2018-10-23 23:29 +0000,792.0,7.0,0.008838,2.0,0.0,4.0,1.0,0.0
4,And then come say hi at next month’s @SPUR_Urb...,2018-10-23 23:09 +0000,532.0,3.0,0.005639,0.0,0.0,1.0,2.0,0.0


Great, that looks a lot better. We also might want to make sure that the time column is a datetime object and not a string.

Darn, looks like it's a string. We should convert it to datetime instead which is a lot easier to work with.

In [54]:
from datetime import datetime
temp = []
for x in range(len(df['time'])):
    # It's in a weird format that datetime doesn't accomodate, so we'll need to remove the last
    # 6 characters of the string and replace it with an appropriate seconds identifier
    temp.append(datetime.strptime(str(df['time'].iloc[x][0:-6] + ':00'), '%Y-%m-%d %H:%M:%S'))
df['time'] = temp

In [55]:
df['time']

0    2018-10-27 18:01:00
1    2018-10-26 14:24:00
2    2018-10-26 02:28:00
3    2018-10-23 23:29:00
4    2018-10-23 23:09:00
             ...        
35   2019-02-04 18:57:00
36   2019-02-02 23:31:00
37   2019-02-02 01:11:00
38   2019-02-01 00:52:00
39   2019-02-01 00:49:00
Name: time, Length: 2249, dtype: datetime64[ns]

Looks good. Let's move on to the most complex column, the text of the tweets themselves. Just as as example take a look at the first tweet.

In [59]:
df['Tweet text'].iloc[0]

"@MTCBATA is looking for a new Executive Director. The last one was in place for 17 years, so this is a BIG deal. We've put together a list of positive, visionary leaders we think are great candidates for the job. Would you add anyone else? Let us know! https://t.co/Syf9exwPTd"

Yikes, that's going to need a lot of cleaning to be usable. A lot more will be done in the pre-processing step, but for now let's just do the basics. Splitting the tweets into individual words, and removing special characters.

In [113]:
# break the tweets up into individual words
tweet_words = []
for x in range(len(df)):
    b = df['Tweet text'].iloc[x].lower()
    c = re.split('\s', b)
    words = list(filter(lambda x: len(re.findall('[a-zA-Z]', x)) != 0, c))
    tweet_words.append(words)

df['tweet words'] = tweet_words

In [114]:
# double check that it worked
df['tweet words']

0     [@mtcbata, is, looking, for, a, new, executive...
1     [ultimate, seamlessness., https://t.co/cdclrg2...
2     [help, nix, prop, save, funding, for, more, se...
3     [it, doesn't, have, to, be, this, way!, let's,...
4     [and, then, come, say, hi, at, next, month’s, ...
                            ...                        
35    [today, is, #transitequityday,, chosen, in, ho...
36    [@alevin, @anniefryman, @scott_wiener, @gillib...
37    [☺️all, the, transit, love, https://t.co/vvzzu...
38    [@ratnaamin, re:seamless, transit, report:, "m...
39    [congrats, @ratnaamin, on, your, new, adventur...
Name: tweet words, Length: 2249, dtype: object

Great. Let's export this cleaned data and move onto the next notebook, which covers EDA.

In [116]:
df.to_csv('/Users/grahamsmith/Documents/SpringboardWork/Seamless_Twitter_Analysis/cleaned tweets.csv')